# weather data

In [13]:
import pandas as pd
import os

train_size = 100 # 1000
test_size = 250 # 250
horizon = 50 # 50

path = os.path.join('results','weather_data',f'train_{train_size}_test_{test_size}_horizon_{horizon}_weather_results.csv')

df = pd.read_csv(path)
# get name of the file without the .csv
name_of_file = os.path.basename(path).split('.')[0]
name_of_file = name_of_file + "_weather"
name_of_file

'train_100_test_250_horizon_50_weather_results_weather'

In [14]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

# Assuming df is your DataFrame
# If you haven't loaded the DataFrame, you can load it using pd.read_csv or other pd functions

# Initialize lists to store metrics
maes = []
rmses = []
mapes = []

# Group by 'RollingOrigin' and calculate metrics for each group
for _, group in df.groupby('RollingOrigin'):
    y_true = group['Actual']
    y_pred = group['TimeGPT']
    
    # Calculate metrics
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mape = mean_absolute_percentage_error(y_true, y_pred)
    
    # Append metrics to lists
    maes.append(mae)
    rmses.append(rmse)
    mapes.append(mape)

# Calculate the mean and standard deviation of the metrics
mae_mean = np.mean(maes)
mae_std = np.std(maes)
rmse_mean = np.mean(rmses)
rmse_std = np.std(rmses)
mape_mean = np.mean(mapes)
mape_std = np.std(mapes)

# Create a DataFrame to store the results
results = pd.DataFrame({
    'Source': [name_of_file],
    'MAE': [mae_mean],
    'MAE_std': [mae_std],
    'RMSE': [rmse_mean],
    'RMSE_std': [rmse_std],
    'MAPE': [mape_mean],
    'MAPE_std': [mape_std]
})
print(results)

# save the results
results.to_csv(os.path.join('error_metrics',f'{name_of_file}.csv'), index=False)

                                              Source       MAE   MAE_std  \
0  train_100_test_250_horizon_50_weather_results_...  1.891755  1.245355   

       RMSE  RMSE_std          MAPE      MAPE_std  
0  2.246882  1.356945  6.680680e+13  1.302837e+14  


# Climate data

In [26]:
import pandas as pd
import os

train_size = 1000
test_size = 50
horizon = 10
model1 = 'timegpt-1'
model2 = 'timegpt-1-long-horizon'

path = os.path.join('results','climate_data', f'train_{train_size}_test_{test_size}_horizon_{horizon}_modeltype_{model1}_climate_results.csv')

df = pd.read_csv(path)
# get name of the file without the .csv
name_of_file = os.path.basename(path).split('.')[0]
name_of_file
df

,date,TimeGPT,Actual,RollingOrigin
0,1953-06-01,0.575735,0.343228,0
1,1953-07-01,0.706922,0.357698,0
2,1953-08-01,0.802228,0.185581,0
3,1953-09-01,0.636237,-0.048259,0
4,1953-10-01,0.464142,0.060318,0
...,...,...,...,...
405,1957-02-01,0.137916,-0.137413,40
406,1957-03-01,0.137567,0.060404,40
407,1957-04-01,0.142735,0.091391,40
408,1957-05-01,0.143780,0.002730,40


In [27]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

# Assuming df is your DataFrame
# If you haven't loaded the DataFrame, you can load it using pd.read_csv or other pd functions

# Initialize lists to store metrics
maes = []
rmses = []
mapes = []

# Group by 'RollingOrigin' and calculate metrics for each group
for _, group in df.groupby('RollingOrigin'):
    y_true = group['Actual']
    y_pred = group['TimeGPT']
    
    # Calculate metrics
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mape = mean_absolute_percentage_error(y_true, y_pred)
    
    # Append metrics to lists
    maes.append(mae)
    rmses.append(rmse)
    mapes.append(mape)

# Calculate the mean and standard deviation of the metrics
mae_mean = np.mean(maes)
mae_std = np.std(maes)
rmse_mean = np.mean(rmses)
rmse_std = np.std(rmses)
mape_mean = np.mean(mapes)
mape_std = np.std(mapes)

# Create a DataFrame to store the results
results = pd.DataFrame({
    'Source': [name_of_file],
    'MAE': [mae_mean],
    'MAE_std': [mae_std],
    'RMSE': [rmse_mean],
    'RMSE_std': [rmse_std],
    'MAPE': [mape_mean],
    'MAPE_std': [mape_std]
})
print(results)

# save the results
results.to_csv(os.path.join('error_metrics',f'{name_of_file}.csv'), index=False)

                                              Source       MAE   MAE_std  \
0  train_1000_test_50_horizon_10_modeltype_timegp...  0.153126  0.064758   

       RMSE  RMSE_std     MAPE  MAPE_std  
0  0.181398  0.065473  2.12848  1.937849  


# t

In [30]:
# load all .csv files in error_metrics folder
import os
import pandas as pd

path = 'error_metrics'
files = os.listdir(path)
files = [file for file in files if file.endswith('.csv')]
files

# combine all csv files into one
combined_csv = pd.concat([pd.read_csv(os.path.join(path, f)) for f in files])
combined_csv.to_csv("combined_csv.csv", index=False, encoding='utf-8-sig')
combined_csv

# save combined results
combined_csv.to_csv(os.path.join('error_metrics','combined_error_results.csv'), index=False)